# Core Analysis

The basic essential bit of code needed to start analysis:

## Imports and settings

In [3]:
# Import 
import os
import pickle
import numpy as np
import pandas as pd
from scipy import ndimage
from scipy.optimize import curve_fit
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import skimage
from skimage.feature import blob_dog, blob_log, blob_doh

import therpy as tp
from breadboard import BreadboardClient

from tqdm import tqdm_notebook as tqdm
from ipywidgets import interact

cst = tp.cst()
cst._fill_physical_constants
cstNa = tp.cst(atom='NaD2')
cstLi = tp.cst(atom='LiD2')

pd.options.mode.chained_assignment = None  # default='warn'

SMALL_SIZE = 12
MEDIUM_SIZE = 15
BIGGER_SIZE = 17
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)   # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
%matplotlib inline

bc = BreadboardClient(config_path='../Processed Data/Fermi3/Database/API_CONFIG_fermi3.json')

# Image settings
settings = {**dict(center_x = int(570), center_y = int(500), width=int(700), height=int(700), subsample=4, 
                    bg_width=0, bg_order=1, bad_light=0, memory_saver=True, 
                    Isat=10.3, time=3, pixel= (30/750) * (6.45 * 1e-6), 
                    od_method='sBL', sigma0=cstNa.sigma0, sigmaf=0.5, fudge=1, )}


In [4]:
### Function defs 
def get_images(images=None, x_var=None, image_func=tp.Image, display=False, force_match=False):
    # This gets images and image parameters from the servers. Uses the clipboard.
    
    # Download the image parameters
    if images==None:
        df = bc.get_images_df_clipboard(xvar=x_var, force_match=force_match)
    else:
        df = bc.get_images_df(images, xvar=x_var, force_match=force_match)
    
    # Download the images
    images = []
    for i, r in tqdm(df.iterrows(), desc='Loading'):
        images.append(image_func(name=r.imagename, lab='fermi3'))
    df['image'] = images

    # sort in time
    df.sort_values(['unixtime', 'imagename'], inplace=True)
    df.reset_index(drop=True, inplace=True)
        
    # Print Information
    if display:
        print('Total number of images   : {}'.format(df.shape[0]))
        print(df.head(2))
    
    # return
    return df


## Load Data

In [6]:
### Get the images from the clipboard:
df = get_images(x_var='holdTime', force_match=True)

### If you have a list of images
# df = get_images(images_to_analyze, x_var='holdTime', force_match=True)


settings = {**dict(center_x = int(570), center_y = int(500), width=int(700), height=int(700), subsample=4, 
                    bg_width=0, bg_order=1, bad_light=0, memory_saver=True, 
                    Isat=10.3, time=3, pixel= (30/750) * (6.45 * 1e-6), 
                    od_method='sBL', sigma0=cstNa.sigma0, sigmaf=0.5, fudge=1, )}

### Apply settings
df['total_atoms']=0
for i, r in tqdm(df.iterrows()):
    r.image.set(**settings)
    df.total_atoms.loc[i] = np.nansum(r.image.app)

PyperclipException: 
    Pyperclip could not find a copy/paste mechanism for your system.
    For more information, please visit
    https://pyperclip.readthedocs.io/en/latest/introduction.html#not-implemented-error
    

In [ ]:
# filtering 
# dfall = df
# df.x = [int(r.imagename[20:]) for i,r in df.iterrows()]
# df = dfall[dfall.Omega==0.8]
# df.x = df.evap3EndMHz

# Sort by x 
# df = df.sort_values('x').reset_index(drop=True)
# df = df.sort_values(['holdTime', 'holdTimeFriend']).reset_index(drop=True)

In [ ]:
# Check out plot crops
def test_func(test=0, ):
    df.image[test].plot_crop()
    print(df.x[test])
    
if df.shape[0] == 1:
    test_func(0)
else:
    interact(test_func, test=(0, df.shape[0]-1, 1));
    

## Plot all Images

In [ ]:

ncols = 10
nrows = max(1,int(np.ceil(len(df)/ncols)))
sizex = 2.5
sizey = 2.5
clim = [-0.1, 20.6]
cmap = 'inferno'

ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=[ncols*sizex, nrows*sizey])[1].flatten()
for i,r in df.iterrows():
#     ax[i].imshow(r.image.app, clim=clim, origin=0, cmap=cmap)
    ax[i].imshow(r.fixed_image, clim=clim, origin=0, cmap=cmap)
    ax[i].text(0.5, 0.9, '{:.1f}'.format(r.x), horizontalalignment='center', verticalalignment='center', transform=ax[i].transAxes, color='w', fontsize=14)
    ax[i].set_axis_off()

for a in ax: a.set_axis_off()

plt.tight_layout(pad=0)
plt.subplots_adjust(wspace=0.02, hspace=0.02)

## Find angle of rotation

In [ ]:
image_angles = []
fixed_images = []
width=int(len(df.iloc[0].image.od)/2)
plotrange=50

xsize = len(df.iloc[0].image.od[(width-plotrange):(width+plotrange),(width-plotrange):(width+plotrange)])
xx = np.linspace(-10, 10, xsize)
yy = np.linspace(-10, 10, xsize)
xx, yy = np.meshgrid(xx, yy)

for i in tqdm(range(len(df))):
    testimage = np.nan_to_num(df.iloc[i].image.od)[(width-plotrange):(width+plotrange),(width-plotrange):(width+plotrange)]
    testimage[testimage>1e10] =0
    # Calculate the moment of inertia tensor
    Ixx = np.sum(testimage*yy*yy)
    Iyy = np.sum(testimage*xx*xx)
    Ixy = np.sum(testimage*xx*yy)
    Iyx = Ixy
    I =np.array( [[Ixx, Ixy], [Iyx, Iyy]])
    evals, evecs = np.linalg.eig(I)
    image_angles.append(180*np.arctan(evecs[np.argmin(evals)][1]/evecs[np.argmin(evals)][0])/np.pi)
    
    fixed_images.append(ndimage.rotate(np.nan_to_num(df.iloc[i].image.od),image_angles[i], reshape=False))
    
df['I_angle'] = image_angles
df['fixed_image'] = fixed_images

# Extended Analysis

Some useful snippets for different lab tasks:

## Averaging over repeats of x

In [ ]:
### df = dataframe containing all of the data. Rows of df can have repeats of df.x eg [0,0,0,1,1,1...]

df_results = pd.DataFrame()
df_results['x'] = np.unique(df.x)
df_results['y_list'] = None
df_results['y_mean'] = None

for k, result in tqdm(df_results.iterrows()):
    
    # Select the rows of the original df that match the current x value
    df_select = df[df.x==result.x]
    y_list = []
    
    for j,r in df_select.iterrows():
        # Process each image to get a y-value
        image = r.image.app
        y = process_image(image)
        y_list.append(y)
        
    # add the y-values to the output dataframe
    df_results.at[k, 'y_list'] = y_list
    df_results.at[k, 'y_mean'] = np.mean(y_list)

# The output df_results contains unique-x rows. 
# Use them to plt.plot(df_results.x, df_results.y_mean) for example
    

## Animation

In [ ]:
# Test a single frame

fig, ax= plt.subplots(1,1, figsize=(7,7))

width=int(len(df.iloc[0].image.od)//2)
plotrange=200

i=0
# image_to_show = ndimage.rotate(df.loc[i,'fixed_image'],0)[(width-plotrange):(width+plotrange),(width-plotrange):(width+plotrange)]
image_to_show = ndimage.rotate(df.loc[i,'image'].od,-5)
image_shown = ax.imshow(image_to_show, cmap='inferno', clim=[0,1.4], origin=0)
title = ax.set_title('t={:.2f}ms'.format(10*df.loc[i,'x']))

ax.set_axis_off()
plt.show()

In [ ]:
dfnew = df
fig, ax= plt.subplots(1,1, figsize=(7,7))

width=int(len(df.iloc[0].image.od)/2)
plotrange=500

i=0
# image_to_show = dfnew.loc[i,'fixed_image'][(width-plotrange):(width+plotrange),(width-plotrange):(width+plotrange)]
image_to_show = df.loc[i,'image'].od
image_shown = ax.imshow(image_to_show, cmap='inferno', clim=[-0.02,1], origin=0)
title = ax.set_title('t={:.2f}ms'.format(df.loc[i,'x']))
ax.set_axis_off()

def animate(i):
#     image_to_show = dfnew.loc[i,'fixed_image'][(width-plotrange):(width+plotrange),(width-plotrange):(width+plotrange)]
    image_to_show = df.loc[i,'image'].od
    image_shown.set_data(image_to_show)
    title.set_text('t={:.2f}ms'.format(df.loc[i,'x']))
    return image_shown, title

ani = animation.FuncAnimation(fig, animate,
                              np.hstack( [
                                        np.repeat(np.arange(len(df)),6),
                                        np.repeat(len(df)-1,15)
                                ]),
                              interval=1, blit=False)


Writer = animation.writers['ffmpeg']
writer = Writer(fps=12, metadata=dict(artist='Me'), bitrate=2800)
ani.save('animations/rotini_2019_06_19_DS8.mp4', writer=writer)

plt.show()

## Vortex Detection

In [ ]:
def vortexfinder(initialOD, plot=False, min_sigma=4, max_sigma=6, threshold=0.3, offset=0.1, block_size=41):

    binary_adaptive = skimage.filters.threshold_local(initialOD, block_size, offset=offset)
    vortexmask = np.array(1.*(binary_adaptive>initialOD))

    blobs = blob_log(vortexmask, min_sigma=min_sigma, max_sigma=max_sigma, threshold=threshold)

    if plot==True:
        f,ax = plt.subplots(ncols=2, figsize=(12,6))
#         ax[0].imshow(vortexmask, origin=0)
        ax[0].imshow(initialOD, origin=0)
        ax[1].imshow(initialOD, origin=0)
        for blob in blobs:
            y,x,r = blob
            c = plt.Circle((x,y), r,lw=2,color='red', fill=False)
            ax[1].add_patch(c)
        for a in ax: a.set_axis_off()
        plt.show()
            
    return blobs

In [ ]:
# for i,r in df.iterrows():
for r in [df.iloc[17]]:
    image = r.image.od
    # image = df.iloc[21].image.od
    image[image==np.inf]=0
    blobs = vortexfinder(np.nan_to_num(image), plot=True, min_sigma=4, max_sigma=10, threshold=0.3, offset=0.1, block_size=41)